In [17]:
from collections import defaultdict
import pandas as pd
import math
import os
import numpy as np

# def itemcf(df_, user_items_dict, use_iif=False):
#     df = df_.copy()
#     user_time_ = df.groupby('user_id')['time'].agg(list).reset_index()  # 引入时间因素
#     user_time_dict = dict(zip(user_time_['user_id'], user_time_['time']))

#     sim_item = {}
#     item_cnt = defaultdict(int)  # 商品被点击次数
#     for user, items in user_items_dict.items():
#         for loc1, item in enumerate(items):
#             item_cnt[item] += 1
#             sim_item.setdefault(item, {})
#             for loc2, relate_item in enumerate(items):
#                 if item == relate_item:
#                     continue
#                 t1 = user_time_dict[user][loc1]  # 点击时间提取
#                 t2 = user_time_dict[user][loc2]
#                 sim_item[item].setdefault(relate_item, 0)
#                 if not use_iif:
#                     if loc1 - loc2 > 0:
#                         sim_item[item][relate_item] += 1 * 1.0  * (0.8**(loc1-loc2-1)) *(
#                                     1 - (t1 - t2) * 10000) / math.log(1 + len(items))  # 逆向
#                     else:
#                         sim_item[item][relate_item] += 1 * 1.0  * (0.8**(loc2-loc1-1)) * (
#                                     1 - (t2 - t1) * 10000) / math.log(1 + len(items))  # 正向
#                 else:
#                     sim_item[item][relate_item] += 1 / math.log(1 + len(items))

#     sim_item_corr = sim_item.copy()  # 引入AB的各种被点击次数
#     for i, related_items in sim_item.items():
#         for j, cij in related_items.items():
#             sim_item_corr[i][j] = cij / ((item_cnt[i] * item_cnt[j]) ** 0.2)

#     return sim_item_corr
def itemcf(df_, user_items_dict, use_iif=False):
    df = df_.copy()
    user_time_ = df.groupby('user_id')['time'].agg(list).reset_index()  # 引入时间因素
    user_time_dict = dict(zip(user_time_['user_id'], user_time_['time']))

    sim_item = {}
    item_cnt = defaultdict(int)  # 商品被点击次数
    for user, items in user_items_dict.items():
        for loc1, item in enumerate(items):
            item_cnt[item] += 1
            sim_item.setdefault(item, {})
            for loc2, relate_item in enumerate(items):
                if item == relate_item:
                    continue
                t1 = user_time_dict[user][loc1]  # 点击时间提取
                t2 = user_time_dict[user][loc2]
                sim_item[item].setdefault(relate_item, 0)
                if not use_iif:
                    if loc1 - loc2 > 0:
                        sim_item[item][relate_item] += 1 * 1.0 * (0.8 ** (loc1 - loc2 - 1)) * (
                                1 - (t1 - t2) * 10000) / math.log(1 + len(items))  # 逆向
                    else:
                        sim_item[item][relate_item] += 1 * 1.0 * (0.8 ** (loc2 - loc1 - 1)) * (
                                1 - (t2 - t1) * 10000) / math.log(1 + len(items))  # 正向
                else:
                    sim_item[item][relate_item] += 1 / math.log(1 + len(items))

    sim_item_corr = sim_item.copy()  # 引入AB的各种被点击次数
    for i, related_items in sim_item.items():
        for j, cij in related_items.items():
            sim_item_corr[i][j] = cij / ((item_cnt[i] * item_cnt[j]) ** 0.2)

    return sim_item_corr


def recall_itemcf(sim_item_corr, user_items_dict, user_id, top_k, item_num):
    '''
    input:item_sim_list, user_item, uid, 500, 50
    # 用户历史序列中的所有商品均有关联商品,整合这些关联商品,进行相似性排序
    '''
#     rank = {}
#     interacted_items = user_items_dict[user_id]
#     interacted_items = interacted_items[::-1]
#     for loc, i in enumerate(interacted_items):
#         for j, wij in sorted(sim_item_corr[i].items(),key=lambda d: d[1], reverse=True)[0:top_k]:
#             if j not in interacted_items:
#                 rank.setdefault(j, 0)
#                 rank[j] += wij * (0.75 ** loc)
    

#     return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]

    rank = {}
#     if user_id not in user_items_dict:  # 如果不存在点击历史，冷启动, 是否存在用户画像，后者根据点击时间使用更短时间段范围的topk填充
#         return []
    interacted_items = user_items_dict[user_id]
    interacted_items = interacted_items[::-1]
    for loc, i in enumerate(interacted_items):
        for j, wij in sorted(sim_item_corr[i].items(), reverse=True)[0:top_k]:
            if j not in interacted_items:
                rank.setdefault(j, 0)
                rank[j] += wij * (0.75 ** loc)

    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]


now_phase = 9
user_path = os.path.expanduser('~')
train_path = os.path.join(user_path, r'kdd\data\underexpose_train')
test_path = os.path.join(user_path, r'kdd\data\underexpose_test')
recom_item = []
whole_click = pd.DataFrame()
click=pd.DataFrame()
times=[]
answer_fname='..\\data\\debias_track_answer_%d.csv'
item_deg = defaultdict(lambda: 0)

for c in range(7, now_phase + 1):
    print('phase:', c)
    click_train = pd.read_csv(train_path + r'\underexpose_train_click-{}.csv'.format(c), header=None,nrows=None,
                              names=['user_id', 'item_id', 'time'],
                              dtype={'user_id':np.int, 'item_id':np.int, 'time':np.float32})
    click_test = pd.read_csv(test_path + r'\underexpose_test_click-{}.csv'.format(c), header=None,nrows=None,
                             names=['user_id', 'item_id', 'time'],
                             dtype={'user_id': np.int, 'item_id': np.int, 'time': np.float32})

    user_test=set(click_test['user_id'])     # 每阶段线下测试集用户集合
    print('len(user_val):', len(user_test))
    user_train=set(click_train['user_id'])-user_test  #每阶段训练集用户集合


    click_train_test = click_train.append(click_test)
    click=click.append(click_train_test)        # 当前阶段以及之前阶段click数据
    click = click.sort_values('time')           # 时间排序
    click= click.drop_duplicates(['user_id', 'item_id', 'time'], keep='last') # 去重


    click['pred'] = click['user_id'].map(lambda x: 'test' if x in user_test else 'train')
    underline_test=click[click['pred']=='test'].drop_duplicates(['user_id'], keep='last') #当前阶段线下测试集click数据
    underline_train=click.append(underline_test).drop_duplicates(keep=False)  #当前阶段以及之前阶段训练集click数据
    assert len(underline_test)+len(underline_train)==len(click)
    
    user_item_ = underline_train.groupby('user_id')['item_id'].agg(list).reset_index()
    # user_items_dict是一个字典，key=user_id,value=用户点击的item列表
    user_items_dict = dict(zip(user_item_['user_id'], user_item_['item_id']))

    top500_click = underline_train['item_id'].value_counts().index[:500].values  # 最热商品
    hot_list = list(top500_click)

    item_sim_list = itemcf(underline_train, user_items_dict, use_iif=False) #计算item的相似度矩阵
    predictions = {}

    for index, user in enumerate(user_test):
        rank_item = recall_itemcf(item_sim_list, user_items_dict, user, 500, 50) 
        item_list = []
        for item in rank_item:
            item_list.append(item[0])
        if len(item_list) < 50:
            for i, item in enumerate(hot_list):
                if item not in item_list:
                    item_list.append(item)
                if len(item_list) == 50:
                    break
        predictions[user] = item_list

    with open('..\\data\\underexpose_submit-%d.csv' % c, 'w') as f:
        temp = str(predictions)
        f.write(temp)

phase: 7
len(user_val): 1797
phase: 8
len(user_val): 1818
phase: 9
len(user_val): 1752


In [18]:
'''
构建线下测试集underline_test
选择每阶段underexpose_test_click.csv中user_id在当前阶段以及之前阶段所有click数据中的最后一次点击作为测试集
'''

from collections import defaultdict
import pandas as pd
import os
import numpy as np
now_phase = 9

user_path = os.path.expanduser('~')
train_path = os.path.join(user_path, r'kdd\data\underexpose_train')
test_path = os.path.join(user_path, r'kdd\data\underexpose_test')
click=pd.DataFrame()
item_deg = defaultdict(lambda: 0) #item_deg是一个字典key=item_id, value=item_id的数量
# answer_fname文件存储每个阶段线下测试集数据，格式phase_id, user_id, item_id, item_deg[item_id]
answer_fname=r'..\\data\\debias_track_answer_%d.csv'


for c in range(7,now_phase + 1):
    print('phase:', c)
    click_train = pd.read_csv(train_path + r'\underexpose_train_click-{}.csv'.format(c), header=None,nrows=None,
                              names=['user_id', 'item_id', 'time'],
                              dtype={'user_id':np.int, 'item_id':np.int, 'time':np.float32})
    click_test = pd.read_csv(test_path + r'\underexpose_test_click-{}.csv'.format(c), header=None,nrows=None,
                             names=['user_id', 'item_id', 'time'],
                             dtype={'user_id': np.int, 'item_id': np.int, 'time': np.float32})

    user_test=set(click_test['user_id'])  # 每阶段线下测试集用户集合
    print('len(user_val):', len(user_test))



    click_train_test = click_train.append(click_test)
    click=click.append(click_train_test)  # 当前阶段以及之前阶段click数据
    click = click.sort_values('time')       # 时间排序
    click= click.drop_duplicates(['user_id', 'item_id', 'time'], keep='last') # 去重


    # 获取当前阶段以及之前阶段item_deg字典，item_id对应的点击次数，用作ndcg_topk_half和hitrate_topk_half统计
    temp_=click
    temp_=temp_.groupby(['item_id'], as_index=False).agg({'user_id':'count'})
    temp_.columns=['item_id','count']
    for item_id,item_count in zip(list(temp_['item_id'].values), list(temp_['count'].values)):
        item_deg[item_id]+=item_count


    click['pred'] = click['user_id'].map(lambda x: 'test' if x in user_test else 'train')
    underline_test=click[click['pred']=='test'].drop_duplicates(['user_id'], keep='last') #当前阶段线下测试集click数据
    underline_train = click.append(underline_test).drop_duplicates(keep=False)  #当前阶段以及之前阶段训练集click数据
    print('underline_test:', underline_test['user_id'].count())
    assert (underline_test['user_id'].count()+underline_train['user_id'].count()==click['user_id'].count())

    with open(answer_fname % c, 'w') as fout:
        for i, row in underline_test.iterrows():
            user_id, item_id, timestamp = (int(row[0]), int(row[1]), row[2])
            assert user_id % 11 == c
            print(c, user_id, item_id, item_deg[item_id], sep=',', file=fout)

phase: 7
len(user_val): 1797
underline_test: 1797
phase: 8
len(user_val): 1818
underline_test: 1818
phase: 9
len(user_val): 1752
underline_test: 1752


In [19]:
'''
评估预测结果
'''
now_phase=9
# answer_fname文件存储每个阶段线下测试集数据，格式phase_id, user_id, item_id, item_deg[item_id]
answer_fname='..\\data\\debias_track_answer_%d.csv'  # 每阶段线下测试集的实际点击
# submit_answer_fname文件保存了线下测试集的user_id的预测点击
submit_answer_fname='..\\data\\underexpose_submit-%d.csv' #每阶段线下测试集的user_id的预测点击
import numpy as np


for topk in [50]: #topk=50,100,...500
#     print('topk={}\n'.format(topk))
    total_ndcg_topk_full = 0.0
    total_ndcg_topk_half = 0.0
    total_hitrate_topk_full = 0.0
    total_hitrate_topk_half = 0.0
    for phase in range(7, now_phase+1):

        with open(submit_answer_fname % phase,'r') as f:
            temp=f.read()
            predictions=eval(temp) # prediction是一个字典key=user_id vakue=预测点击的商品列表

        answers={}
        with open(answer_fname % phase, 'r') as fin:
            for line in fin:
                line = [int(x) for x in line.split(',')]
                phase_id, user_id, item_id, item_degree = line
                assert user_id % 11 == phase_id
                answers[user_id] = (item_id, item_degree)

        list_item_degress = []
        for user_id in answers:
            item_id, item_degree = answers[user_id]
            list_item_degress.append(item_degree)
        list_item_degress.sort()
        median_item_degree = list_item_degress[len(list_item_degress) // 2] #商品统计次数中值

        num_cases_full = 0.0
        ndcg_topk_full = 0.0
        ndcg_topk_half = 0.0
        num_cases_half = 0.0
        hitrate_topk_full = 0.0
        hitrate_topk_half = 0.0
        
        for user in answers:
            item_id, item_degree = answers[user]
            item_list=predictions[user]
            rank = 0
            while rank < topk and item_list[rank] != item_id:
                rank += 1
            num_cases_full += 1.0
            if rank < topk:
                ndcg_topk_full += 1.0 / np.log2(rank + 2.0)
                hitrate_topk_full += 1.0
            if item_degree <= median_item_degree:
                num_cases_half += 1.0
                if rank < topk:
                    ndcg_topk_half += 1.0 / np.log2(rank + 2.0)
                    hitrate_topk_half += 1.0

        ndcg_topk_full /= num_cases_full
        ndcg_topk_half /= num_cases_half
        hitrate_topk_full /= num_cases_full
        hitrate_topk_half /= num_cases_half

        # 每阶段指标相加
        total_ndcg_topk_full += ndcg_topk_full
        total_ndcg_topk_half += ndcg_topk_half
        total_hitrate_topk_full += hitrate_topk_full
        total_hitrate_topk_half += hitrate_topk_half

        print('phase={}\ntopk={}\nndcg_topk_full={}\nndcg_topk_half={}\nhitrate_topk_full={}\nhitrate_topk_half={}\n'.format(
            phase, topk, ndcg_topk_full, ndcg_topk_half, hitrate_topk_full, hitrate_topk_half))
        
        with open('..\\data\\score_detail.csv', 'a+') as f:
            print(phase, topk, ndcg_topk_full, ndcg_topk_half, hitrate_topk_full, hitrate_topk_half,
                  sep=',',file=f)

#     print('topk={}\n, ndcg_topk_full={}\n, ndcg_topk_half={}\n, hitrate_topk_full={}\n, hitrate_topk_half={}\n'.format(
#           topk, total_ndcg_topk_full, total_ndcg_topk_half, total_hitrate_topk_full, total_hitrate_topk_half))
#     with open('data\\score', 'w') as f:
#         print(topk, total_ndcg_topk_full, total_ndcg_topk_half, total_hitrate_topk_full, total_hitrate_topk_half,
#               sep=',', file=f)

phase=7
topk=50
ndcg_topk_full=0.047978067180391125
ndcg_topk_half=0.03318225544417068
hitrate_topk_full=0.11463550361713967
hitrate_topk_half=0.0841995841995842

phase=8
topk=50
ndcg_topk_full=0.044060337230176556
ndcg_topk_half=0.03599781346913106
hitrate_topk_full=0.10286028602860287
hitrate_topk_half=0.09044193216855087

phase=9
topk=50
ndcg_topk_full=0.03660348894446413
ndcg_topk_half=0.02705710827198607
hitrate_topk_full=0.07705479452054795
hitrate_topk_half=0.05622932745314223



In [14]:
'''
评估预测结果
'''
now_phase=9
# answer_fname文件存储每个阶段线下测试集数据，格式phase_id, user_id, item_id, item_deg[item_id]
answer_fname='..\\data\\debias_track_answer_%d.csv'  # 每阶段线下测试集的实际点击
# submit_answer_fname文件保存了线下测试集的user_id的预测点击
submit_answer_fname='.underexpose_submit2020-06-07_23_16_30.csv' #每阶段线下测试集的user_id的预测点击
import numpy as np


for topk in [50]: #topk=50,100,...500
#     print('topk={}\n'.format(topk))
    total_ndcg_topk_full = 0.0
    total_ndcg_topk_half = 0.0
    total_hitrate_topk_full = 0.0
    total_hitrate_topk_half = 0.0
    with open(submit_answer_fname, 'r') as f:
            for line in f:
                line=[int(x) for x in line.split(',')]
                user=line[0]
                list_items=line[1:]
                predictions[user]=list_items
            
    for phase in range(7, now_phase+1):
        answers={}
        with open(answer_fname % phase, 'r') as fin:
            for line in fin:
                line = [int(x) for x in line.split(',')]
                phase_id, user_id, item_id, item_degree = line
                assert user_id % 11 == phase_id
                answers[user_id] = (item_id, item_degree)

        list_item_degress = []
        for user_id in answers:
            item_id, item_degree = answers[user_id]
            list_item_degress.append(item_degree)
        list_item_degress.sort()
        median_item_degree = list_item_degress[len(list_item_degress) // 2] #商品统计次数中值

        num_cases_full = 0.0
        ndcg_topk_full = 0.0
        ndcg_topk_half = 0.0
        num_cases_half = 0.0
        hitrate_topk_full = 0.0
        hitrate_topk_half = 0.0
        
        for user in answers:
            item_id, item_degree = answers[user]
            item_list=predictions[user]
            rank = 0
            while rank < topk and item_list[rank] != item_id:
                rank += 1
            num_cases_full += 1.0
            if rank < topk:
                ndcg_topk_full += 1.0 / np.log2(rank + 2.0)
                hitrate_topk_full += 1.0
            if item_degree <= median_item_degree:
                num_cases_half += 1.0
                if rank < topk:
                    ndcg_topk_half += 1.0 / np.log2(rank + 2.0)
                    hitrate_topk_half += 1.0

        ndcg_topk_full /= num_cases_full
        ndcg_topk_half /= num_cases_half
        hitrate_topk_full /= num_cases_full
        hitrate_topk_half /= num_cases_half

        # 每阶段指标相加
        total_ndcg_topk_full += ndcg_topk_full
        total_ndcg_topk_half += ndcg_topk_half
        total_hitrate_topk_full += hitrate_topk_full
        total_hitrate_topk_half += hitrate_topk_half

        print('phase={}\ntopk={}\nndcg_topk_full={}\nndcg_topk_half={}\nhitrate_topk_full={}\nhitrate_topk_half={}\n'.format(
            phase, topk, ndcg_topk_full, ndcg_topk_half, hitrate_topk_full, hitrate_topk_half))
        
        with open('..\\data\\score_detail.csv', 'a+') as f:
            print(phase, topk, ndcg_topk_full, ndcg_topk_half, hitrate_topk_full, hitrate_topk_half,
                  sep=',',file=f)

#     print('topk={}\n, ndcg_topk_full={}\n, ndcg_topk_half={}\n, hitrate_topk_full={}\n, hitrate_topk_half={}\n'.format(
#           topk, total_ndcg_topk_full, total_ndcg_topk_half, total_hitrate_topk_full, total_hitrate_topk_half))
#     with open('data\\score', 'w') as f:
#         print(topk, total_ndcg_topk_full, total_ndcg_topk_half, total_hitrate_topk_full, total_hitrate_topk_half,
#               sep=',', file=f)

phase=7
topk=50
ndcg_topk_full=0.0
ndcg_topk_half=0.0
hitrate_topk_full=0.0
hitrate_topk_half=0.0

phase=8
topk=50
ndcg_topk_full=0.0
ndcg_topk_half=0.0
hitrate_topk_full=0.0
hitrate_topk_half=0.0

phase=9
topk=50
ndcg_topk_full=0.0
ndcg_topk_half=0.0
hitrate_topk_full=0.0
hitrate_topk_half=0.0

